In [78]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [79]:
drop_tables = """
-- =========================
-- DROP TABLES
-- =========================

DROP TABLE IF EXISTS Event_Medikament CASCADE;
DROP TABLE IF EXISTS Arzt_Event CASCADE;
DROP TABLE IF EXISTS Event CASCADE;
DROP TABLE IF EXISTS Behandlung CASCADE;
DROP TABLE IF EXISTS Medikament CASCADE;
DROP TABLE IF EXISTS Bett CASCADE;
DROP TABLE IF EXISTS Reserve CASCADE;
DROP TABLE IF EXISTS Pflegekraft CASCADE;
DROP TABLE IF EXISTS Arzt CASCADE;
DROP TABLE IF EXISTS Patient CASCADE;
DROP TABLE IF EXISTS Raum CASCADE;
DROP TABLE IF EXISTS Station CASCADE;
DROP TABLE IF EXISTS Krankenhaus CASCADE;
"""



In [80]:
cur.execute(drop_tables)
conn.commit()
print("Alle Tabellen wurden gelöscht.")

Alle Tabellen wurden gelöscht.


In [81]:
create_tables = """
CREATE TABLE Krankenhaus (
    KH_ID INT PRIMARY KEY,
    Strasse VARCHAR(100) NOT NULL,
    Hausnummer VARCHAR(10) NOT NULL,
    Postleitzahl VARCHAR(10) NOT NULL,
    Stadt VARCHAR(50) NOT NULL,
    Name VARCHAR(100) NOT NULL,
    CHECK (Postleitzahl ~ '^[0-9]{4}$')
);

CREATE TABLE Station (
    Station_ID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL,
    Kapazitaet INT NOT NULL,
    KH_ID INT NOT NULL,
    CHECK (Kapazitaet > 0),
    FOREIGN KEY (KH_ID) REFERENCES Krankenhaus(KH_ID)
);

CREATE TABLE Raum (
    Raumnummer INT PRIMARY KEY,
    Station_ID INT NOT NULL,
    Art VARCHAR(50),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Pflegekraft (
    Pflege_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Station_ID INT NOT NULL,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Patient (
    Patient_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Versicherungsnummer VARCHAR(30) UNIQUE NOT NULL,
    Geburtsdatum DATE NOT NULL,
    Strasse VARCHAR(100),
    Hausnummer VARCHAR(10),
    Postleitzahl VARCHAR(10),
    Stadt VARCHAR(50),
    Elektronisches_Patientendossier_ID VARCHAR(50) UNIQUE,
    Station_ID INT,
    CHECK (Postleitzahl IS NULL OR Postleitzahl ~ '^[0-9]{4}$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Bett (
    Bett_ID INT PRIMARY KEY,
    Status VARCHAR(20) NOT NULL,
    Patient_ID INT,
    Station_ID INT NOT NULL,
    CHECK (Status IN ('frei', 'belegt')),
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Arzt (
    Arzt_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    ist_extern BOOLEAN NOT NULL,
    Station_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Reserve (
    Reserve_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Arzt_ID INT,
    Station_ID INT,
    Pflegekraft_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID),
    FOREIGN KEY (Pflegekraft_ID) REFERENCES Pflegekraft(Pflege_ID)
);

CREATE TABLE Behandlung (
    Behandlungs_ID INT PRIMARY KEY,
    abgeschlossen BOOLEAN NOT NULL,
    Beschreibung VARCHAR(255),
    Start_Datum DATE NOT NULL,
    End_Datum DATE,
    Patient_ID INT NOT NULL,
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    CHECK (End_Datum IS NULL OR End_Datum >= Start_Datum)
);

CREATE TABLE Medikament (
    Medikament_ID INT PRIMARY KEY,
    Wirkstoff VARCHAR(100),
    Name VARCHAR(100) NOT NULL   
);

CREATE TABLE Event (
    Event_ID INT PRIMARY KEY,
    Uhrzeit TIME NOT NULL,
    Datum DATE NOT NULL,
    RaumID INT NOT NULL,
    Behandlung_ID INT NOT NULL,
    FOREIGN KEY (RaumID) REFERENCES Raum(Raumnummer),
    FOREIGN KEY (Behandlung_ID) REFERENCES Behandlung(Behandlungs_ID)
);

CREATE TABLE Arzt_Event (
    Arzt_ID INT NOT NULL,
    Event_ID INT NOT NULL,
    PRIMARY KEY (Arzt_ID, Event_ID),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID)
);

CREATE TABLE Event_Medikament (
    Event_ID INT NOT NULL,
    Medikament_ID INT NOT NULL,
    PRIMARY KEY (Event_ID, Medikament_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Medikament_ID) REFERENCES Medikament(Medikament_ID)
);
"""
cur.execute(create_tables)
conn.commit()


In [82]:
import random
from faker import Faker

fake = Faker("de_CH")

def phone(i):
    return f"+41 79 {i:06d}"

STATION_NAMES = [
    "Chirurgie",
    "Innere Medizin",
    "Pädiatrie",
    "Notfall",
    "Intensivstation"
]

RAUM_ARTEN = [
    "Operationssaal",
    "Untersuchungsraum",
    "Patientenzimmer",
    "Behandlungsraum"
]

MEDIKAMENTE = [
    ("Paracetamol", "Paracetamol"),
    ("Ibuprofen", "Ibuprofen"),
    ("Aspirin", "Acetylsalicylsäure"),
    ("Amoxicillin", "Amoxicillin"),
    ("Morphin", "Morphin"),
    ("Insulin", "Insulin"),
    ("Metformin", "Metformin"),
    ("Cortison", "Hydrocortison"),
    ("Omeprazol", "Omeprazol"),
    ("Salbutamol", "Salbutamol")
]

# =========================
# Krankenhaus
# =========================
for kh_id in range(1, 4):
    cur.execute("""
        INSERT INTO Krankenhaus
        (KH_ID, Strasse, Hausnummer, Postleitzahl, Stadt, Name)
        VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        kh_id,
        fake.street_name(),
        fake.building_number(),
        f"{1000 + kh_id}",
        fake.city(),
        f"Kantonsspital {kh_id}"
    ))
conn.commit()

# =========================
# Station
# =========================
station_id = 1
for kh_id in range(1, 4):
    for name in STATION_NAMES:
        cur.execute("""
            INSERT INTO Station
            (Station_ID, Name, Kapazitaet, KH_ID)
            VALUES (%s,%s,%s,%s)
        """, (
            station_id,
            name,
            40,
            kh_id
        ))
        station_id += 1
conn.commit()

# =========================
# Patient
# =========================
for pid in range(1, 201):
    cur.execute("""
        INSERT INTO Patient
        (Patient_ID, Vorname, Nachname, Versicherungsnummer,
         Geburtsdatum, Strasse, Hausnummer, Postleitzahl,
         Stadt, Elektronisches_Patientendossier_ID, Station_ID)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (
        pid,
        fake.first_name(),
        fake.last_name(),
        f"CH{pid:08d}",
        fake.date_of_birth(minimum_age=0, maximum_age=95),
        fake.street_name(),
        fake.building_number(),
        f"{1000 + pid % 50}",
        fake.city(),
        f"EPD-{pid}",
        ((pid - 1) % (station_id - 1)) + 1
    ))
conn.commit()

# =========================
# Arzt
# =========================
arzt_id = 1
for i in range(1, 51):
    cur.execute("""
        INSERT INTO Arzt
        (Arzt_ID, Telefonnummer, Vorname, Nachname, ist_extern, Station_ID)
        VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        arzt_id,
        phone(i),
        fake.first_name(),
        fake.last_name(),
        i % 5 == 0,
        ((i - 1) % (station_id - 1)) + 1
    ))
    arzt_id += 1
conn.commit()

# =========================
# Pflegekraft
# =========================
pflege_id = 1
for i in range(1, 81):
    cur.execute("""
        INSERT INTO Pflegekraft
        (Pflege_ID, Telefonnummer, Vorname, Nachname, Station_ID)
        VALUES (%s,%s,%s,%s,%s)
    """, (
        pflege_id,
        phone(200 + i),
        fake.first_name(),
        fake.last_name(),
        ((i - 1) % (station_id - 1)) + 1
    ))
    pflege_id += 1
conn.commit()

# =========================
# Reserve
# =========================
for rid in range(1, 31):
    cur.execute("""
        INSERT INTO Reserve
        (Reserve_ID, Telefonnummer, Arzt_ID, Station_ID, Pflegekraft_ID)
        VALUES (%s,%s,%s,%s,%s)
    """, (
        rid,
        phone(500 + rid),
        ((rid - 1) % (arzt_id - 1)) + 1,
        ((rid - 1) % (station_id - 1)) + 1,
        ((rid - 1) % (pflege_id - 1)) + 1
    ))
conn.commit()

# =========================
# Bett
# =========================
bett_id = 1
for sid in range(1, station_id):
    for _ in range(10):
        cur.execute("""
            INSERT INTO Bett
            (Bett_ID, Status, Patient_ID, Station_ID)
            VALUES (%s,%s,%s,%s)
        """, (
            bett_id,
            "frei",
            None,
            sid
        ))
        bett_id += 1
conn.commit()

# =========================
# Raum
# =========================
raum_id = 1
for sid in range(1, station_id):
    for art in RAUM_ARTEN:
        cur.execute("""
            INSERT INTO Raum
            (Raumnummer, Station_ID, Art)
            VALUES (%s,%s,%s)
        """, (
            raum_id,
            sid,
            art
        ))
        raum_id += 1
conn.commit()

# =========================
# Medikament
# =========================
for mid, (name, wirkstoff) in enumerate(MEDIKAMENTE, start=1):
    cur.execute("""
        INSERT INTO Medikament
        (Medikament_ID, Wirkstoff, Name)
        VALUES (%s,%s,%s)
    """, (
        mid,
        wirkstoff,
        name
    ))
conn.commit()

# =========================
# Behandlung
# =========================
for bid in range(1, 201):
    cur.execute("""
        INSERT INTO Behandlung
        (Behandlungs_ID, abgeschlossen, Beschreibung,
         Start_Datum, End_Datum, Patient_ID)
        VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        bid,
        bid % 2 == 0,
        "Standardbehandlung",
        fake.date_between("-6m", "-2m"),
        fake.date_between("-2m", "today"),
        bid
    ))
conn.commit()

# =========================
# Event
# =========================
for eid in range(1, 501):
    cur.execute("""
        INSERT INTO Event
        (Event_ID, Uhrzeit, Datum, RaumID, Behandlung_ID)
        VALUES (%s,%s,%s,%s,%s)
    """, (
        eid,
        fake.time(),
        fake.date_between("-3m", "today"),
        ((eid - 1) % (raum_id - 1)) + 1,
        ((eid - 1) % 200) + 1
    ))
conn.commit()

# =========================
# Arzt_Event
# =========================
for eid in range(1, 501):
    cur.execute("""
        INSERT INTO Arzt_Event
        (Arzt_ID, Event_ID)
        VALUES (%s,%s)
    """, (
        ((eid - 1) % (arzt_id - 1)) + 1,
        eid
    ))
conn.commit()

# =========================
# Event_Medikament
# =========================
for eid in range(1, 501):
    for mid in range(1, 3):
        cur.execute("""
            INSERT INTO Event_Medikament
            (Event_ID, Medikament_ID)
            VALUES (%s,%s)
        """, (
            eid,
            ((eid + mid) % len(MEDIKAMENTE)) + 1
        ))
conn.commit()


In [83]:
conn.rollback()
